1. EDA (Explotary Data Analysis)
2. Hand-made Logistic Regression
3. Logistic Regression with sklearn

****EDA (Explotary Data Analysis)****

In [31]:
# libraries
import numpy as np                
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
# read csv
data = pd.read_csv("../input/weatherAUS.csv")

In [33]:
# drop the unnecessary columns
data.drop(["Date", "Location", "WindGustDir", "WindDir9am", "WindDir3pm"], axis=1, inplace=True)

We have to get rid of **NaN**, **YES** and **NO**.

In [34]:
# convert "NO" to 0 and "YES" to 1
data.RainTomorrow = [ 1 if each == "Yes" else 0 for each in data.RainTomorrow ]
data.RainToday = [ 1 if each == "Yes" else 0 for each in data.RainToday ]

# convert Nan to 0
data = data.fillna(0)

data.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,13.4,22.9,0.6,0.0,0.0,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,0,0.0,0
1,7.4,25.1,0.0,0.0,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,0.0,0.0,17.2,24.3,0,0.0,0
2,12.9,25.7,0.0,0.0,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,0.0,2.0,21.0,23.2,0,0.0,0
3,9.2,28.0,0.0,0.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,0.0,0.0,18.1,26.5,0,1.0,0
4,17.5,32.3,1.0,0.0,0.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0.2,0


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 19 columns):
MinTemp          142193 non-null float64
MaxTemp          142193 non-null float64
Rainfall         142193 non-null float64
Evaporation      142193 non-null float64
Sunshine         142193 non-null float64
WindGustSpeed    142193 non-null float64
WindSpeed9am     142193 non-null float64
WindSpeed3pm     142193 non-null float64
Humidity9am      142193 non-null float64
Humidity3pm      142193 non-null float64
Pressure9am      142193 non-null float64
Pressure3pm      142193 non-null float64
Cloud9am         142193 non-null float64
Cloud3pm         142193 non-null float64
Temp9am          142193 non-null float64
Temp3pm          142193 non-null float64
RainToday        142193 non-null int64
RISK_MM          142193 non-null float64
RainTomorrow     142193 non-null int64
dtypes: float64(17), int64(2)
memory usage: 20.6 MB


In [36]:
# y is results. x is features without "RainTomorrow" and 'nan'
y = data.RainTomorrow.values
x = data.drop(["RainTomorrow"], axis=1)


****Hand-made Logistic Regression****

In [37]:
# normalization
x = ((x - np.min(x))/(np.max(x) - np.min(x))).values

In [38]:
# train/test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

x_train shape:  (18, 113754)
x_test shape:  (18, 28439)
y_train shape:  (113754,)
y_test shape:  (28439,)


In [39]:
# initialize weight and bias
def initialize_w_and_b(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

In [40]:
# sigmoid func
def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

In [41]:
# forward & backward propagation func
def forward_backward_propagation(w,b,x_train,y_train):
    # forward
    z = np.dot(w.T,x_train)+b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]
    
    # backward
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost, gradients

In [42]:
# updating
def update(w,b,x_train,y_train,learning_rate,num_iterations):
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(num_iterations):
        cost,gradients = forward_backward_propagation(w,b,x_train, y_train)
        cost_list.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        param = { "weight": w, "bias": b }
        """if i % 50 == 0:
            index.append(i)
            cost_list2.append(cost)
            
    
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("iteation")
    plt.ylabel("cost")
    plt.show()"""  # wanna see graph, delete """
    
    return param, gradients, cost_list

In [43]:
# prediction
def predict(w,b,x_test):
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if z[0,i] <= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
    return Y_prediction

In [44]:
# logistic regression
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate, num_iterations):
    # initialize
    dimension = x_train.shape[0]   # that is 30
    w,b = initialize_w_and_b(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate, num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"], x_test)
    
    # print test error
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))

now, lets call logistic regression func

In [50]:
logistic_regression(x_train, y_train, x_test, y_test, learning_rate = 1, num_iterations = 300)

test accuracy: 82.06336369070642 %


****Logistic Regression with sklearn****

In [46]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train.T, y_train.T)
print("test accuracy: {}".format(lr.score(x_test.T, y_test.T)))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


test accuracy: 0.9018601216639123
